In [1]:
from urllib.parse import urlparse
from sqlalchemy import create_engine
import pandas as pd


#dbserver_sampleの中身を書き換えて、ファイル名をdbserver.txtに変更
#dbserver.txtからdbのpathを読み込む

db_path = open('dbserver_sample.txt', 'r').read()

In [2]:
#dbに接続し、テーブルから取ってきたデータをdataframeに変換
from sqlalchemy.sql import text
from datetime import datetime, date, timezone, timedelta

url = urlparse(db_path) 

conn = create_engine('mysql://{user}:{password}@{host}:{port}/{database}?charset=utf8'.format(host = url.hostname, port=url.port, user = url.username, password= url.password, database = url.path[1:]),encoding='utf-8')

table='raw_from_api'
# sql_read=text("select * from qiita_db.raw_from_api where likes_count = {};".format(640))
#sql_read=text("select * from qiita_db.raw_from_api where created_at = '{}';".format('2019-01-01 10:53:44'))
sql_read=text("select * from qiita_db.raw_from_api where created_at between'{}' and '{}';".format('2019-01-01 00:00:00','2019-02-01 00:00:00'))

df = pd.read_sql(sql_read, conn)

In [3]:
df.head(5)

,id,article_id,title,created_at,updated_at,likes_count,comments_count,tags_str,user_id,user_permanent_id,url,html,tokens
0,469421,a0ae89bb2ff08c50d4ec,Mac上でWindowsのコマンドラインアプリをビルドする,2019-01-01 09:29:37,2019-01-01 09:29:37,0,0,"C,Mac,MacOSX,wine,MinGW",ossan-buhibuhi,101524,https://qiita.com/ossan-buhibuhi/items/a0ae89b...,<p>Chicken Schemeがファイルパス関係でどう動作するか確認したかったんだけど、...,Chicken Schemeがファイルパス関係でどう動作するか確認したかったんだけど、Win...
1,469420,298d3bf877b73ce50612,Webアプリ開発を試したいJSライブラリ,2019-01-01 09:33:15,2019-01-01 09:33:15,3,0,JavaScript,giogiokuma,60856,https://qiita.com/giogiokuma/items/298d3bf877b...,<p>自分が試したいJSライブラリをメモ（2019）<br>\n<strong><a hre...,自分が試したいJSライブラリをメモ（2019）\n**[howler.js](https:/...
2,469419,2bad5f7dda7c35afc9fa,Qiitaで自分のContributinランキングが分かる最強システムの秘密とは‥？ RT ...,2019-01-01 09:55:56,2019-01-01 09:56:15,0,0,"Qiita,Twitter",YumaInaura,89618,https://qiita.com/YumaInaura/items/2bad5f7dda7...,"<p><a href=""https://mogya.github.io/qiitaRank/...",https://mogya.github.io/qiitaRank/\n\n>エンジニアラン...
3,469418,d0fb65f6800253598e45,JSの日付の条件分岐について,2019-01-01 10:08:06,2019-01-01 10:08:06,0,0,JavaScript,suzuki_san,317636,https://qiita.com/suzuki_san/items/d0fb65f6800...,<p>JavaScriptで１月１日から表示する項目をDateを使って出し分けようとしたが、...,JavaScriptで１月１日から表示する項目をDateを使って出し分けようとしたが、うまく...
4,469417,5590cdde6f61ef729ffd,@Qiita のTwitter通知。ツイート内容にアカウント名を埋め込むだけじゃダメ‥？,2019-01-01 10:09:30,2019-01-01 10:09:30,0,0,"Qiita,Twitter",YumaInaura,89618,https://qiita.com/YumaInaura/items/5590cdde6f6...,"\n<h1>\n<span id=""アカウント名は入らない"" class=""fragment...",\n# アカウント名は入らない\n\n![image](https://user-image...


In [4]:
len(df)

8248

In [8]:
import pandas as pd
from typing import Tuple
import lxml.html
import re

def get_string(html_text):
    html = lxml.html.fromstring(html_text)
    remove_tags = ('.//style', './/script', './/noscript')
    for remove_tag in remove_tags:
        for tag in html.findall(remove_tag):
            tag.drop_tree()
            # ここでの削除は元の変数tに反映されます。
    
    codeframe_list = []
    lang_list = []
    # コードの削除
    for tag in html.findall(".//div[@class='code-frame']"):
        codeframe_list.append(tag.text_content())
        lang_list.append(tag.attrib["data-lang"])
        tag.drop_tree()

    atext_list = []
    ahref_list = []
    # href　リンクの削除
    for tag in html.cssselect('a'):
        if tag.text is not None:
            atext_list.append(tag.text)
        if tag.get('href') is not None:
            ahref_list.append(tag.get('href'))
        tag.drop_tree()
        
    code_list = []
    # 一行コードの削除    
    for cc in  html.cssselect('code'):
        if cc.text is not None:
            code_list.append(cc.text)
        cc.drop_tree()
    
    text = html.text_content().strip('\n')

    return  pd.Series(["".join(text.split('\n')), ",".join(codeframe_list), ",".join(lang_list), ",".join(code_list), ",".join(atext_list), ",".join(ahref_list)], index=['text', 'code-frame', 'lang', 'code', 'a-text', 'a-href'])



In [10]:
df_text = pd.DataFrame(columns=['id', 'text', 'code-frame', 'lang', 'code', 'a-text', 'a-href'])
for index, row in df.iterrows():
    seri = get_string(row['html'])
    ss = pd.concat([seri, row])
#     seri['id'] = row['id']
    df_text = df_text.append(ss, ignore_index=True)

In [11]:
df_text.head(5)

,id,text,code-frame,lang,code,a-text,a-href,article_id,comments_count,created_at,html,likes_count,tags_str,title,tokens,updated_at,url,user_id,user_permanent_id
0,469421,Chicken Schemeがファイルパス関係でどう動作するか確認したかったんだけど、Win...,,,,,,a0ae89bb2ff08c50d4ec,0.0,2019-01-01 09:29:37,<p>Chicken Schemeがファイルパス関係でどう動作するか確認したかったんだけど、...,0.0,"C,Mac,MacOSX,wine,MinGW",Mac上でWindowsのコマンドラインアプリをビルドする,Chicken Schemeがファイルパス関係でどう動作するか確認したかったんだけど、Win...,2019-01-01 09:29:37,https://qiita.com/ossan-buhibuhi/items/a0ae89b...,ossan-buhibuhi,101524.0
1,469420,自分が試したいJSライブラリをメモ（2019）オーディオスプライトによる制御可能 > オーデ...,var sound = new Howl({\n src:['sound.mp3']\n}...,"text,text,text,text,text,text,text,text",,"howler.js,「howler.js」https://howlerjs.com/,jPL...","https://howlerjs.com/,https://howlerjs.com/,ht...",298d3bf877b73ce50612,0.0,2019-01-01 09:33:15,<p>自分が試したいJSライブラリをメモ（2019）<br>\n<strong><a hre...,3.0,JavaScript,Webアプリ開発を試したいJSライブラリ,自分が試したいJSライブラリをメモ（2019）\n**[howler.js](https:/...,2019-01-01 09:33:15,https://qiita.com/giogiokuma/items/298d3bf877b...,giogiokuma,60856.0
2,469419,"エンジニアランキングvQiitaのcontributions値から,日本でどのくらいのエンジ...",,,,"https://mogya.github.io/qiitaRank/,@mogya,http...","https://mogya.github.io/qiitaRank/,/mogya,#%E3...",2bad5f7dda7c35afc9fa,0.0,2019-01-01 09:55:56,"<p><a href=""https://mogya.github.io/qiitaRank/...",0.0,"Qiita,Twitter",Qiitaで自分のContributinランキングが分かる最強システムの秘密とは‥？ RT ...,https://mogya.github.io/qiitaRank/\n\n>エンジニアラン...,2019-01-01 09:56:15,https://qiita.com/YumaInaura/items/2bad5f7dda7...,YumaInaura,89618.0
3,469418,JavaScriptで１月１日から表示する項目をDateを使って出し分けようとしたが、うまく...,,,,,,d0fb65f6800253598e45,0.0,2019-01-01 10:08:06,<p>JavaScriptで１月１日から表示する項目をDateを使って出し分けようとしたが、...,0.0,JavaScript,JSの日付の条件分岐について,JavaScriptで１月１日から表示する項目をDateを使って出し分けようとしたが、うまく...,2019-01-01 10:08:06,https://qiita.com/suzuki_san/items/d0fb65f6800...,suzuki_san,317636.0
4,469417,アカウント名は入らないQiitaは ユーザーごとに Twitter のアカウントを知っている...,,,,"https://github.com/YumaInaura/YumaInaura/,http...",#%E3%82%A2%E3%82%AB%E3%82%A6%E3%83%B3%E3%83%88...,5590cdde6f61ef729ffd,0.0,2019-01-01 10:09:30,"\n<h1>\n<span id=""アカウント名は入らない"" class=""fragment...",0.0,"Qiita,Twitter",@Qiita のTwitter通知。ツイート内容にアカウント名を埋め込むだけじゃダメ‥？,\n# アカウント名は入らない\n\n![image](https://user-image...,2019-01-01 10:09:30,https://qiita.com/YumaInaura/items/5590cdde6f6...,YumaInaura,89618.0


In [16]:
df_text = df_text.drop(["html", "created_at", "updated_at", "tokens", "user_id", "user_permanent_id", "comments_count", "code-frame", "code", "a-text", "a-href"], axis=1)

In [17]:
df_text.head(3)

,id,text,lang,article_id,likes_count,tags_str,title,url
0,469421,Chicken Schemeがファイルパス関係でどう動作するか確認したかったんだけど、Win...,,a0ae89bb2ff08c50d4ec,0.0,"C,Mac,MacOSX,wine,MinGW",Mac上でWindowsのコマンドラインアプリをビルドする,https://qiita.com/ossan-buhibuhi/items/a0ae89b...
1,469420,自分が試したいJSライブラリをメモ（2019）オーディオスプライトによる制御可能 > オーデ...,"text,text,text,text,text,text,text,text",298d3bf877b73ce50612,3.0,JavaScript,Webアプリ開発を試したいJSライブラリ,https://qiita.com/giogiokuma/items/298d3bf877b...
2,469419,"エンジニアランキングvQiitaのcontributions値から,日本でどのくらいのエンジ...",,2bad5f7dda7c35afc9fa,0.0,"Qiita,Twitter",Qiitaで自分のContributinランキングが分かる最強システムの秘密とは‥？ RT ...,https://qiita.com/YumaInaura/items/2bad5f7dda7...


In [26]:
# 欠損値の数と割合を求める
def missingval_table(df):
    mv_df = pd.DataFrame(df.isnull().sum())
    mv_df = mv_df.rename(columns={0:'null count'})
    mv_df['null count(%)'] = mv_df['null count']/len(df)*100
    mv_df['variation'] = df.nunique().values
    mv_df['variation(%)'] = df.nunique().values/len(df)*100
    return mv_df

In [29]:
import numpy as np
df1 = df_text.replace("", np.nan, regex=True)
missingval_table(df1)

,null count,null count(%),variation,variation(%)
id,0,0.000000,8248,100.000000
text,18,0.218235,8205,99.478661
lang,2247,27.242968,2984,36.178468
article_id,0,0.000000,8248,100.000000
likes_count,0,0.000000,210,2.546072
tags_str,0,0.000000,6119,74.187682
title,0,0.000000,8241,99.915131
url,0,0.000000,8248,100.000000


In [30]:
df_text.to_csv("2019-01-01-content.csv")